<a href="https://colab.research.google.com/github/anushapalisetty/Intel-Image-Classification/blob/main/Densnet_GitIntel_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import cv2
import pandas as pd




In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Image Generator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Add data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,horizontal_flip=True,shear_range=0.2,  
    zoom_range=0.2, validation_split=0.1)

"""train_datagen = ImageDataGenerator(rescale = 1./255,
	    rotation_range=15,
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0.01,
      zoom_range=[0.1, 1.5],
      horizontal_flip=True,
      fill_mode='nearest',
      zca_whitening=True,
      brightness_range=[0.1, 2]
)"""


train_generator=train_datagen.flow_from_directory(
     '/content/drive/My Drive/Colab Notebooks/DL_Project_Data/seg_train/seg_train/',
      target_size=(150,150),
      batch_size=64,
      class_mode='sparse',
      seed=2209,
      subset='training'
    
)

validation_generator=train_datagen.flow_from_directory(
      '/content/drive/My Drive/Colab Notebooks/DL_Project_Data/seg_train/seg_train/',
      target_size=(150,150),
      batch_size=64,
      class_mode='sparse',
      seed=2209,
      subset='validation'
    
)

test_datagen = ImageDataGenerator(rescale = 1./255.)
test_generator = test_datagen.flow_from_directory(
    '/content/drive/My Drive/Colab Notebooks/DL_Project_Data/seg_test/seg_test/', 
    target_size=(150,150),
    batch_size=32,
    class_mode='sparse',
    seed=2209
) 

test1_datagen = ImageDataGenerator(rescale = 1./255.)
test1_generator = test1_datagen.flow_from_directory(
    '/content/drive/My Drive/Colab Notebooks/DL_Project_Data/seg_test/seg_test/', 
    target_size=(150,150),
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=2209
) 

Found 12667 images belonging to 6 classes.
Found 1406 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


# **DenseNet121**

In [5]:
from keras.layers import *
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
import numpy as np 
from keras.applications.densenet import DenseNet121
import keras


base_model1=DenseNet121(include_top=False, weights= 'imagenet',  input_shape=(150,150,3), pooling='avg')
base_model1.trainable = False

x = Dense(256, activation='relu')(base_model1.output)
x = Dropout(0.5)(x)
x = Dense(6, activation='softmax')(x)

transfer_model1 = Model(base_model1.input, x) 
transfer_model1.compile(optimizer =keras.optimizers.Adam(lr=0.0001), 
              loss = 'sparse_categorical_crossentropy', 
              metrics = ['accuracy'])

29089792/29084464 [==============================] - 0s 0us/step


In [6]:
import tensorflow as tf
reduce = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, mode='auto')
history2 = transfer_model1.fit_generator(
            train_generator,
            steps_per_epoch=int(12667 /64),
            epochs=60,
            validation_data=validation_generator,
            validation_steps=int(1406 /64),
            callbacks=[reduce],
            verbose=1)
transfer_model1.save('/content/drive/My Drive/Colab Notebooks/DL_Project_Data/model_Dense_Git_3.hd5')


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/60
197/197 [==============================] - 3083s 16s/step - loss: 1.0193 - accuracy: 0.6313 - val_loss: 0.4593 - val_accuracy: 0.8467
Epoch 2/60
197/197 [==============================] - 99s 504ms/step - loss: 0.4995 - accuracy: 0.8158 - val_loss: 0.3581 - val_accuracy: 0.8772
Epoch 3/60
197/197 [==============================] - 97s 493ms/step - loss: 0.4234 - accuracy: 0.8404 - val_loss: 0.3275 - val_accuracy: 0.8854
Epoch 4/60
197/197 [==============================] - 96s 489ms/step - loss: 0.3909 - accuracy: 0.8574 - val_loss: 0.3235 - val_accuracy: 0.8884
Epoch 5/60
197/197 [==============================] - 97s 494ms/step - loss: 0.3679 - accuracy: 0.8656 - val_loss: 0.3023 - val_accuracy: 0.8914
Epoch 6/60
197/197 [==============================] - 96s 490ms/step - loss: 0.3501 - accuracy: 0.8724 - val_loss: 0.2948 - val_accuracy: 0.8996
Epoch 7/60
197/197 [==============================] - 

In [7]:
transfer_model1.save('/content/drive/My Drive/Colab Notebooks/DL_Project_Data/model_Dense_Git_pro3.hd5')

import pandas as pd
df_cnn = pd.DataFrame(history.history)      
# saving the dataframe  
df_cnn.to_csv('/content/drive/My Drive/Colab Notebooks/DL_Project_Data/model_Dense_Git_3_History.csv')  

INFO:tensorflow:Assets written to: /content/drive/My Drive/Colab Notebooks/DL_Project_Data/model_Dense_Git_3.hd5/assets


NameError: ignored

In [ ]:
accuracy1=transfer_model1.evaluate_generator(test_generator, steps=len(test_generator)//32,verbose=2)
print('Accuracy of the model on the test set: ',accuracy1[1])

In [ ]:
#Evaluating Accuracy and Loss of the model
%matplotlib inline
acc=history2.history['acc']
val_acc=history2.history['val_acc']
loss=history2.history['loss']
val_loss=history2.history['val_loss']

epochs=range(len(acc)) #No. of epochs

#Plot training and validation accuracy per epoch
import matplotlib.pyplot as plt
plt.plot(epochs,acc,'b',label='Training Accuracy')
plt.plot(epochs,val_acc,'r',label='Validation Accuracy')
plt.legend()
plt.figure()

#Plot training and validation loss per epoch
plt.plot(epochs,loss,'b',label='Training Loss')
plt.plot(epochs,val_loss,'r',label='Validation Loss')
plt.legend()
plt.show()

# **ResNet50**

In [ ]:
from keras.layers import *
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
import numpy as np 
from keras.applications.resnet50 import ResNet50
import keras


base_model2=ResNet50(include_top=False, weights= 'imagenet',  input_shape=(150,150,3), pooling='avg')
base_model2.trainable = False

x = Dense(512, activation='relu')(base_model2.output)
x = Dropout(0.5)(x)
x = Dense(6, activation='softmax')(x)

transfer_model2 = Model(base_model2.input, x) 
transfer_model2.compile(optimizer =keras.optimizers.SGD(lr=0.0001), 
              loss = 'sparse_categorical_crossentropy', 
              metrics = ['accuracy'])

In [ ]:
import tensorflow as tf
reduce = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, mode='auto')
history3 = transfer_model2.fit_generator(
            train_generator,
            steps_per_epoch=int(12632/64),
            epochs=50,
            validation_data=validation_generator,
            validation_steps=int(1402/64),
            callbacks=[reduce],
            verbose=1)

In [ ]:
accuracy2=transfer_model2.evaluate_generator(test_generator, steps=len(test_generator)//32,verbose=2)
print('Accuracy of the model on the test set: ',accuracy2[1])

In [ ]:
#Evaluating Accuracy and Loss of the model
%matplotlib inline
acc=history3.history['acc']
val_acc=history3.history['val_acc']
loss=history3.history['loss']
val_loss=history3.history['val_loss']

epochs=range(len(acc)) #No. of epochs

#Plot training and validation accuracy per epoch
import matplotlib.pyplot as plt
plt.plot(epochs,acc,'b',label='Training Accuracy')
plt.plot(epochs,val_acc,'r',label='Validation Accuracy')
plt.legend()
plt.figure()

#Plot training and validation loss per epoch
plt.plot(epochs,loss,'b',label='Training Loss')
plt.plot(epochs,val_loss,'r',label='Validation Loss')
plt.legend()
plt.show()